In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM
np.set_printoptions(precision=3, suppress=True)

In [ ]:
df=pd.read_csv("traf.csv")
dataset=df[['quantity','pop','city','lat','capacity','container','price','brand']]
x=dataset
x = pd.concat([x.drop('city',axis=1),pd.get_dummies(x.city)],axis=1)
x = pd.concat([x.drop('container',axis=1),pd.get_dummies(x.container)],axis=1)
x = pd.concat([x.drop('brand',axis=1),pd.get_dummies(x.brand)],axis=1)

In [ ]:
x.head()

,quantity,pop,lat,capacity,price,Addis Ababa,Amhara,Oromia,SPPR,Tigray,glass,plastic,Coca-Cola,Fanta-Orange,Fanta-pineapple,Schewpps,Sprite
0,13280.0,672130.0,37.97945,500,0.96,1,0,0,0,0,1,0,1,0,0,0,0
1,6727.0,672130.0,37.97945,1,2.86,1,0,0,0,0,0,1,1,0,0,0,0
2,9848.0,672130.0,37.97945,352,0.87,1,0,0,0,0,0,1,1,0,0,0,0
3,20050.0,672130.0,37.97945,500,1.00,1,0,0,0,0,1,0,0,1,0,0,0
4,25696.0,672130.0,37.97945,352,0.39,1,0,0,0,0,0,1,0,1,0,0,0


In [ ]:
cols=list(x)[0:12]

In [ ]:
cols

['quantity',
 'pop',
 'lat',
 'capacity',
 'price',
 'Addis Ababa',
 'Amhara',
 'Oromia',
 'SPPR',
 'Tigray',
 'glass',
 'plastic']

In [47]:
type(training_set)

numpy.ndarray

In [48]:
type(training_set_scaled)

numpy.ndarray

In [53]:
dataset_train

,quantity,pop,lat,capacity,price,Addis Ababa,Amhara,Oromia,SPPR,Tigray,glass,plastic
0,13280.0,672130.0,37.97945,500.0,0.96,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6727.0,672130.0,37.97945,1.0,2.86,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9848.0,672130.0,37.97945,352.0,0.87,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,20050.0,672130.0,37.97945,500.0,1.00,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,25696.0,672130.0,37.97945,352.0,0.39,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6371,33201.0,665871.0,37.96245,1.0,1.02,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6372,46971.0,144302.0,39.63689,352.0,0.47,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6373,47708.0,168501.0,38.24444,500.0,1.02,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6374,27115.0,353001.0,40.64361,1.0,1.34,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
dataset_train = x[cols].astype(str)
for i in cols:
    for j in range(0, len(dataset_train)):
        dataset_train[i][j] = dataset_train[i][j].replace(',', '')

dataset_train = dataset_train.astype(float)

# Using multiple features (predictors)
training_set = dataset_train.values

print('Shape of training set == {}.'.format(training_set.shape))
training_set

Shape of training set == (6376, 12).


array([[ 13280.   , 672130.   ,     37.979, ...,      0.   ,      1.   ,
             0.   ],
       [  6727.   , 672130.   ,     37.979, ...,      0.   ,      0.   ,
             1.   ],
       [  9848.   , 672130.   ,     37.979, ...,      0.   ,      0.   ,
             1.   ],
       ...,
       [ 47708.   , 168501.   ,     38.244, ...,      0.   ,      1.   ,
             0.   ],
       [ 27115.   , 353001.   ,     40.644, ...,      1.   ,      0.   ,
             1.   ],
       [ 30413.   , 353001.   ,     40.644, ...,      1.   ,      1.   ,
             0.   ]])

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)

sc_predict = StandardScaler()
sc_predict.fit_transform(training_set[:, 0:1])

array([[-0.902],
       [-1.268],
       [-1.094],
       ...,
       [ 1.022],
       [-0.129],
       [ 0.056]])

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_past, dataset_train.shape[1]-1)))

# Adding 2nd LSTM layer
model.add(LSTM(units=10, return_sequences=False))

# Adding Dropout
model.add(Dropout(0.25))

# Output layer
model.add(Dense(units=1, activation='linear'))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')

In [15]:
history = model.fit(X_train, y_train, shuffle=True, epochs=5, validation_split=0.2,verbose=1)

Epoch 1/5
35/35 [==============================] - 273s 8s/step - loss: 0.7027 - val_loss: 0.3893
Epoch 2/5
35/35 [==============================] - 263s 8s/step - loss: 0.6287 - val_loss: 0.4073
Epoch 3/5
35/35 [==============================] - 269s 8s/step - loss: 0.5632 - val_loss: 0.3681
Epoch 4/5
35/35 [==============================] - 262s 7s/step - loss: 0.5316 - val_loss: 0.3777
Epoch 5/5
35/35 [==============================] - 262s 8s/step - loss: 0.4922 - val_loss: 0.3918


In [20]:
type(y_train)

numpy.ndarray

In [44]:
y_train[3]

array([0.519])

In [40]:
c=X_train[3]

In [41]:
d = c.reshape(1, 5000, 11)

In [42]:
e=model.predict(d)

1/1 [==============================] - 1s 620ms/step


In [45]:
predicted_data=e

In [46]:
scaler = StandardScaler()
scaler.fit(training_set[:, 0:1])  # Assuming the first column was scaled during training

# Reverse the scaling
reversed_data = scaler.inverse_transform(predicted_data.reshape(-1, 1))

# Flatten the reversed data array if necessary
reversed_data = reversed_data.flatten()

print(reversed_data)

[24240.025]


In [13]:
X_train = []
y_train = []

n_future = 1   # Number of days we want top predict into the future
n_past = 5000    # Number of past days we want to use to predict the future

for i in range(n_past, len(training_set_scaled) - n_future +1):
    X_train.append(training_set_scaled[i - n_past:i, 0:dataset_train.shape[1] - 1])
    y_train.append(training_set_scaled[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

print('X_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))

X_train shape == (1376, 5000, 11).
y_train shape == (1376, 1).


In [52]:
type(y_train)

numpy.ndarray